In [1]:
import imageio
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapsOnImage
from PIL import Image
from numpy import asarray
import os


In [7]:
Path = "/media/sebastian/TOSHIBA EXT/SebastianM/DataSet_Final/Primer_augmentation/"
listnames_img = os.listdir(Path+"Img")
listnames_mask = os.listdir(Path+"Mask")

def names_img(listnames_img,Path):
    listnames_img_arr = []
    for idx,vars in enumerate(listnames_img):
        nameImage = Path + "Img/" + vars
        listnames_img_arr.append(nameImage)
    return listnames_img_arr

def names_mask(listnames_mask,Path):
    listnames_mask_arr = []
    for idx,vars in enumerate(listnames_mask):
        nameImage = Path + 'Mask/' + vars
        listnames_mask_arr.append(nameImage)
    return listnames_mask_arr
    

In [5]:
# Define our augmentation pipeline.
seq1 = iaa.Sequential([
    iaa.Sharpen((0.0, 1.0)),       # sharpen the image
    iaa.Affine(rotate=(-5,5),scale=(1.2,1.5)), 
    iaa.Fliplr(0.5),
    iaa.GaussianBlur(sigma=(0.0,1.5))
], random_order=True)

seq2 = iaa.Sequential([
    iaa.Sharpen((0.0, 1.0)),       # sharpen the image
    iaa.Affine(rotate=(-20,20),scale=(1.5,1.7)), 
    iaa.Fliplr(0.5),
    iaa.GaussianBlur(sigma=(0.0,1.5))
], random_order=True)

In [9]:
def augmenter(listnames_img_arr,listnames_mask_arr,Path,numImgPerImg=15):
    #numImgPerImg -> Number of result images for one image.
    allNames = list(zip(listnames_img_arr,listnames_mask_arr))
    contP = 0
    contS = 0
    #img_contP_contS.png
    #mask_contP_contS.png
    for img,mask in allNames:
        img = asarray(Image.open(img))
        mask = asarray(Image.open(mask))
        mask = SegmentationMapsOnImage(mask, shape=img.shape)
        mask = mask.resize(sizes = (480,640))
        
        for i in range(numImgPerImg):
            img_aug_i, mask_aug_i = seq1(image=img, segmentation_maps=mask)
            if i >= int(numImgPerImg/2):
                img_aug_i, mask_aug_i = seq2(image=img, segmentation_maps=mask)
            
            img_aug_i = Image.fromarray(img_aug_i)
            mask_aug_i = Image.fromarray(mask_aug_i.get_arr())

            img_aug_i.save(Path +'x_aug/'+'img'+'_'+str(contP)+'_'+str(contS)+'.png')
            mask_aug_i.save(Path +'y_aug/'+'mask'+'_'+str(contP)+'_'+str(contS)+'.png')
            contS += 1
        contP += 1

listnames_img_arr = names_img(listnames_img,Path)
listnames_mask_arr = names_mask(listnames_mask,Path)        
augmenter(listnames_img_arr,listnames_mask_arr,Path)        